In [1]:
import requests
import lxml
import time
import random

import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome(executable_path='C:\\chromedriver.exe')

driver.get('https://scholar.google.com/citations?view_op=view_org&hl=en&org=11304740246128990681')



In [ ]:
#scarping name list
names=[]
ProfileLinks=[]
topic = []
citations = []
titles = []

URL = 'https://scholar.google.com/citations?view_op=view_org&hl=en&org=11304740246128990681'

while True:
    text = driver.page_source
    soup = BeautifulSoup(text, 'html.parser')
    people = soup.find_all('h3', {"class": "gs_ai_name"})
    for person in people:
        names.append(person.text)
        ProfileLinks.append('https://scholar.google.com/' + person.find_all('a')[0].get('href'))
    t = soup.find_all('div', {"class": "gs_ai_aff"})
    titles += [i.text for i in t]
    c = soup.find_all('div', {"class": "gs_ai_cby"})
    if c:
        citations += [int(i.text[9:]) for i in c]
    f = soup.find_all('div', {"class": "gs_ai_int"})
    for fields in f:
        field = fields.find_all('a', {"class": "gs_ai_one_int"})
        topics = ''
        for t in field:
            topics += t.text + ', '
        topics = topics[:-2]
        topic.append(topics)
    button = driver.find_element(By.XPATH,"//button[@class='gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx']")
    time.sleep(random.uniform(1.5, 2))
    button.click()
    time.sleep(3)
    NewURL = driver.current_url
    if NewURL == URL:
        break
    URL = NewURL
    driver.get(URL)
    
driver.close()

PeopleGMU = pd.DataFrame()
PeopleGMU['names'] = names
PeopleGMU['ProfileLinks'] = ProfileLinks
PeopleGMU['topics'] = topic
PeopleGMU['citations'] = citations
PeopleGMU['titles'] = titles

PeopleGMU

In [474]:
gmu1 = gmu1.drop(columns='Unnamed: 0')
gmu2 = gmu2.drop(columns='Unnamed: 0')
gmu1.head()

,names,ProfileLinks,topics,citations,titles
0,Lance Liotta,https://scholar.google.com//citations?hl=en&us...,"cancer, bioengineering, proteomics",149941,Professor George Mason University
1,Zoltan Acs,https://scholar.google.com//citations?hl=en&us...,"Innovation, entrepreneurship, economic geograp...",70244,George Mason University
2,Joseph A Maxwell,https://scholar.google.com//citations?hl=en&us...,"qualitative and mixed methods research, causat...",68653,George Mason University
3,Emanuel Petricoin,https://scholar.google.com//citations?hl=en&us...,"personalized medicine, cancer, cell sgnaling, ...",50137,George Mason University
4,Raja Parasuraman,https://scholar.google.com//citations?hl=en&us...,"Human Factors, Attention, Automation, Brain St...",49532,"Professor of Psychology, George Mason University"


In [38]:
#scraping publications
text = driver.page_source
soup = BeautifulSoup(text, 'html.parser')

objects = soup.find_all('tr', {"class": "gsc_a_tr"})

base = ['' for i in range(len(objects))]
pubs = pd.DataFrame()
pubs['titles'] = base
pubs['authors'] = base
pubs['date'] = base
pubs['source'] = base
pubs['descriptions'] = base
pubs['citations'] = base

for i, obj in enumerate(objects):
    pubs.loc[i, 'titles'] = obj.find_all('a')[0].text.replace('\xa0', ' ')
    time.sleep(random.uniform(0.75, 1.5))
    driver.find_element_by_link_text(pubs.loc[i, 'titles']).click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    p = soup.find_all('div', {"class": "gsc_oci_value"})
    pubs.loc[i, 'authors'] = soup.find_all('div', {"class": "gsc_oci_value"})[0].text
    pubs.loc[i, 'date'] = soup.find_all('div', {"class": "gsc_oci_value"})[1].text
    pubs.loc[i, 'source'] = soup.find_all('div', {"class": "gsc_oci_field"})[2].text + ' ' + soup.find_all('div', {"class": "gsc_oci_value"})[2].text
    if soup.find_all('div', {"class": "gsh_csp"}):
        pubs.loc[i, 'descriptions'] = soup.find_all('div', {"class": "gsh_csp"})[0].text
    elif soup.find_all('div', {"class": "gsh_small"}):
        pubs.loc[i, 'descriptions'] = soup.find_all('div', {"class": "gsh_small"})[0].text
    if soup.find_all('div', {"style": "margin-bottom:1em"}):
        pubs.loc[i, 'citations'] = soup.find_all('div', {"style": "margin-bottom:1em"})[0].text[9:]
    time.sleep(random.uniform(1, 2))
    driver.back()
pubs

,titles,authors,date,source,descriptions,citations
0,Introduction to software testing,"Paul Ammann, Jeff Offutt",2016/12/13,PublisherCambridge University Press,This extensively classroom-tested text takes a...,2384
1,Constraint-based automatic test data generation,"RA DeMillo, A. Jefferson Offutt",1991/9,"JournalSoftware Engineering, IEEE Transactions on",This paper presents a new technique for automa...,1169
2,MuJava: an automated class mutation system,"Yu‐Seung Ma, Jeff Offutt, Yong Rae Kwon",2005/6,"JournalSoftware Testing, Verification and Reli...",Several module and class testing techniques ha...,870
3,An Experimental Determination of Sufficient Mu...,"Jeff Offutt, Gregg Rothermel, Roland H Untch, ...",1996/4,JournalACM Transactions on software engineerin...,"Mutation testing is a technique, originally pr...",859
4,Generating tests from UML specifications,"Jeff Offutt, Aynur Abdurazik",1999,Journal«UML»’99—The Unified Modeling Language,Although most industry testing of complex soft...,665
...,...,...,...,...,...,...
355,General Co-Chairs,"Carlo Ghezzi, Yuxi Fu, Shaoying Liu, Jim Woodc...",Organizing Committee Page 1 xv Organizing Comm...,"Scholar articlesGeneral Co-ChairsC Ghezzi, Y F...",,
356,TDD: 1st International Workshop on Test-Driven...,"Laurie Williams, Jeff Offutt, Michael Feathers...",TDD: 1st International Workshop on Test-Driven...,Scholar articlesTDD: 1st International Worksho...,,
357,Mutation: 5th International Workshop on Mutati...,"Lydie du Bousquet, Jeremy Bradbury, Gordon Fra...",Mutation: 5th International Workshop on Mutati...,Scholar articlesMutation: 5th International Wo...,,
358,A Message from the Mutation 2007 Programme Co-...,"John Clark, Jeff Offutt",We are very pleased with the outstanding submi...,Scholar articlesA Message from the Mutation 20...,We are very pleased with the outstanding submi...,


In [451]:
#name list
driver = webdriver.Chrome(executable_path='C:\\chromedriver.exe')

driver.get(personURL)

button = driver.find_element(By.XPATH,"//button[@id='gsc_bpf_more']")

while True:
    if not button.is_enabled():
        break
    button.click()
    time.sleep(random.uniform(0.15, 1))
    
text = driver.page_source
soup = BeautifulSoup(text, 'html.parser')

objects = soup.find_all('tr', {"class": "gsc_a_tr"})

titles=[]
links=[]
for obj in objects:
    combined = str(obj.find_all('a', {"class": "gsc_a_at"}))[27:-5]
    titles.append(combined[combined.find('"')+2:])
    links.append('https://scholar.google.com' + combined[:combined.find('"')].replace('&amp;', '&').replace('&oe=ASCII', ''))

pubs = pd.DataFrame()
pubs['titles'] = titles
pubs['links'] = links

pubs[['descriptions', 'URL']] = ''

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

for i in range(len(pubs)):
    time.sleep(random.uniform(1, 2))
    text = requests.get(pubs.links[i], headers=headers).text
    soup = BeautifulSoup(text, 'html.parser')
    if soup.find_all('div', {"class": "gsh_csp"}):
        pubs.loc[i, 'descriptions'] = soup.find_all('div', {"class": "gsh_csp"})[0].text
    elif soup.find_all('div', {"class": "gsh_small"}):
        pubs.loc[i, 'descriptions'] = soup.find_all('div', {"class": "gsh_small"})[0].text
    if soup.find_all('a', {"class": "gsc_oci_title_link"}):
        pubs.loc[i, 'URL'] = soup.find_all('a', {"class": "gsc_oci_title_link"})[0].get('href')
    else:
        pubs.loc[i, 'URL'] = ' '
    
    

driver.close()    
pubs

,titles,links,descriptions,URL
0,Introduction to software testing,https://scholar.google.com/citations?view_op=v...,This extensively classroom-tested text takes a...,https://books.google.com/books?hl=en&lr=&id=58...
1,Constraint-based automatic test data generation,https://scholar.google.com/citations?view_op=v...,This paper presents a new technique for automa...,http://citeseerx.ist.psu.edu/viewdoc/download?...
2,MuJava: an automated class mutation system,https://scholar.google.com/citations?view_op=v...,Several module and class testing techniques ha...,https://onlinelibrary.wiley.com/doi/abs/10.100...
3,An Experimental Determination of Sufficient Mu...,https://scholar.google.com/citations?view_op=v...,"Mutation testing is a technique, originally pr...",https://dl.acm.org/doi/pdf/10.1145/227607.227610
4,Generating tests from UML specifications,https://scholar.google.com/citations?view_op=v...,Although most industry testing of complex soft...,https://link.springer.com/chapter/10.1007/3-54...
...,...,...,...,...
355,General Co-Chairs,https://scholar.google.com/citations?view_op=v...,,https://www.computer.org/csdl/pds/api/csdl/pro...
356,TDD: 1st International Workshop on Test-Driven...,https://scholar.google.com/citations?view_op=v...,,https://ieeexplore.ieee.org/abstract/document/...
357,Mutation: 5th International Workshop on Mutati...,https://scholar.google.com/citations?view_op=v...,,https://www.computer.org/csdl/proceedings-arti...
358,A Message from the Mutation 2007 Programme Co-...,https://scholar.google.com/citations?view_op=v...,We are very pleased with the outstanding submi...,https://ieeexplore.ieee.org/abstract/document/...


In [39]:
driver.close() 